In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import BertTokenizer, BertModel, AdamW;

import torch;
import torch.nn as nn;
import torch.nn.functional as F;
import torch.optim as optim;
from torch.utils.data import Dataset, DataLoader;

In [ ]:
import os;
import os.path;

In [ ]:
import numpy as np;
import matplotlib.pyplot as plt;

import statistics;

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu');
print(device);

cuda


In [ ]:
"""
    Using bert model for downstream classification task.
"""

class BERTClassifier(nn.Module):
    def __init__(self, model_name, num_classes):
        super(BERTClassifier, self).__init__();
        self.bert = BertModel.from_pretrained(model_name);
        self.dropout = nn.Dropout(p=0.2);
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes);

    def forward(self, input_ids, attention_mask):
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask);
        output = self.dropout(output.pooler_output);
        output = self.fc(output);
        return output;

In [ ]:
bert_model_name = 'bert-base-uncased';
num_classes = 3;
max_length = 512;
batch_size = 16;
epochs = 3;
learning_rate = 1e-5;

In [ ]:
tokenizer = BertTokenizer.from_pretrained(bert_model_name);

In [ ]:
class Dataset(nn.Module):
    def __init__(self,path_x,path_y,max_length):

        self.max_length = max_length;

        if not os.path.exists(path_x):
            raise FileNotFoundError(path_x);
        if not os.path.exists(path_y):
            raise FileNotFoundError(path_y);

        with open(path_x,'r') as f:
            self.x = f.readlines();

        with open(path_y,'r') as f:
            self.y = f.readlines();

        if len(self.x) != len(self.y):
            raise ValueError("x and y must have same length");

    def __len__(self):
        return len(self.x);

    def __getitem__(self,idx):
        x = self.x[idx];
        y = self.y[idx];
        output = tokenizer(x, max_length=self.max_length, padding='max_length', truncation=True, return_tensors='pt');
        return {
            'input_ids': output['input_ids'].flatten(),
            'attention_mask': output['attention_mask'].flatten(),
            'label' : torch.tensor(int(self.y[idx]))
        };

In [ ]:
train_dataset = Dataset('/content/drive/MyDrive/Data/DATA/rev_trainset_cmb.txt',
                        '/content/drive/MyDrive/Data/DATA/y_train.txt',max_length);

val_dataset = Dataset('/content/drive/MyDrive/Data/DATA/rev_valset_cmb.txt',
                      '/content/drive/MyDrive/Data/DATA/y_val.txt',max_length);

train_loader = DataLoader(train_dataset,batch_size=batch_size,shuffle=True);
val_loader = DataLoader(val_dataset,batch_size=batch_size,shuffle=True);

In [ ]:
model = BERTClassifier(bert_model_name, num_classes);
model = model.to(device);
optimizer = AdamW(model.parameters(), lr=learning_rate);
criterion = nn.CrossEntropyLoss();

In [ ]:
store_best_path = "/content/drive/MyDrive/PARAM/";

In [ ]:
epochs = 5;

train_loss = [];
val_loss = [];
train_acc = [];
val_acc = [];

for epoch in range(epochs):

    train_loss_epoch = [];
    val_loss_epoch = [];
    train_acc_epoch = [];
    val_acc_epoch = [];

    model.train();
    for batch in train_loader:
        optimizer.zero_grad();

        input_ids = batch['input_ids'].to(device);
        attention_mask = batch['attention_mask'].to(device);
        y = batch['label'].to(device);

        output = model(input_ids,attention_mask);
        loss = criterion(output,y);
        loss.backward();
        optimizer.step();

        train_loss_epoch.append(loss.item());

        accuracy = torch.argmax(output,dim=-1).view(-1) == y.view(-1);
        train_acc_epoch.append((torch.sum(accuracy) / len(accuracy)).item());


    model.eval();
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device);
            attention_mask = batch['attention_mask'].to(device);
            y = batch['label'].to(device);

            output = model(input_ids,attention_mask);
            loss = criterion(output,y);

            val_loss_epoch.append(loss.item());

            accuracy = torch.argmax(output,dim=-1).view(-1) == y.view(-1);
            val_acc_epoch.append((torch.sum(accuracy) / len(accuracy)).item());


    print(f"Epoch[{epoch}][{epochs}] : Training Loss :{statistics.mean(train_loss_epoch)}, Validation Loss :{statistics.mean(val_loss_epoch)}, \
Training Accuracy :{statistics.mean(train_acc_epoch)}, Validation Accuracy :{statistics.mean(val_acc_epoch)}");

    if len(val_loss) == 0 or statistics.mean(val_loss_epoch) < min(val_loss):
        torch.save(model.state_dict(),store_best_path + 'bert_rouge.pt');

    train_loss.append(statistics.mean(train_loss_epoch));
    val_loss.append(statistics.mean(val_loss_epoch));
    train_acc.append(statistics.mean(train_acc_epoch));
    val_acc.append(statistics.mean(val_acc_epoch));

Epoch[0][5] : Training Loss :1.009646703004837, Validation Loss :0.9755233526229858, Training Accuracy :0.454375, Validation Accuracy :0.4525
Epoch[1][5] : Training Loss :0.9425327682495117, Validation Loss :0.9520699071884156, Training Accuracy :0.521875, Validation Accuracy :0.5225
Epoch[2][5] : Training Loss :0.8549542713165283, Validation Loss :0.9754091727733613, Training Accuracy :0.6015625, Validation Accuracy :0.50875
Epoch[3][5] : Training Loss :0.6947817783057689, Validation Loss :1.0235264921188354, Training Accuracy :0.7190625, Validation Accuracy :0.53125
Epoch[4][5] : Training Loss :0.48276638604700567, Validation Loss :1.196674988269806, Training Accuracy :0.8278125, Validation Accuracy :0.51375
